In [106]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## Pandas and Basic Data manipulation
The basic structure in pands is the DataFrame.  This is very similar to R's dataframe, and in fact was modeled after it.  You can think of this object as being like a spreadsheet, with columns and rows and header labels.  In pands you can create a DataFrame from many different data structures.  

Let's look as some json (javascript object notation) obtained from the web.

In [107]:
import json # a library for dealing with json in python
import urllib2 # a library with convenient functions for urls

url = "http://jsonplaceholder.typicode.com/posts" 
response = urllib2.urlopen(url)
if response.code != 200:
    print "Something went wrong"

# Now that we have a response from the website, let's read the data and convert the 
# json string into a python data structure for us to use.
raw_data = response.read()
structured_data = json.loads(raw_data)

In [108]:
print "The number of objects received is %d" % len(structured_data)
sample = structured_data[0]
print "Each object is a %s, this is a python data structure that is a hash map.  It has keys and values.\n" % str(type(sample))
for k in sample:
    print "Key: {}\nValue: {}\n".format(k, sample[k])

The number of objects received is 100
Each object is a <type 'dict'>, this is a python data structure that is a hash map.  It has keys and values.

Key: body
Value: quia et suscipit
suscipit recusandae consequuntur expedita et cum
reprehenderit molestiae ut ut quas totam
nostrum rerum est autem sunt rem eveniet architecto

Key: userId
Value: 1

Key: id
Value: 1

Key: title
Value: sunt aut facere repellat provident occaecati excepturi optio reprehenderit



### Now let's use pandas to load this up as a spreadsheet-like object and examine it

In [111]:
df = pd.DataFrame(structured_data)

# What is the shape of the dataframe?
print "Shape: ", df.shape
# Look at the top 5 rows to see what we are dealing with.
df.head()

Shape:  (100, 4)


,body,id,title,userId
0,quia et suscipit\nsuscipit recusandae consequu...,1,sunt aut facere repellat provident occaecati e...,1
1,est rerum tempore vitae\nsequi sint nihil repr...,2,qui est esse,1
2,et iusto sed quo iure\nvoluptatem occaecati om...,3,ea molestias quasi exercitationem repellat qui...,1
3,ullam et saepe reiciendis voluptatem adipisci\...,4,eum et est occaecati,1
4,repudiandae veniam quaerat sunt sed\nalias aut...,5,nesciunt quas odio,1


Notice that the left hand side is numbering the rows.  This is the index.
We can set the index to be another one of the values.  For many of the options
in pandas, we will have the option of returning a modified object, or modifying
the existing object.  e.g.
```python
df2 = df.set_index('id')                  # create a new dataframe, df2, while df is unaffected
df.set_index('id', inplace=True)          # df is changed, notice this method returns None
```

In [114]:
df2 = df.set_index('id')
df2.head()

,body,title,userId
id,,,
1,quia et suscipit\nsuscipit recusandae consequu...,sunt aut facere repellat provident occaecati e...,1
2,est rerum tempore vitae\nsequi sint nihil repr...,qui est esse,1
3,et iusto sed quo iure\nvoluptatem occaecati om...,ea molestias quasi exercitationem repellat qui...,1
4,ullam et saepe reiciendis voluptatem adipisci\...,eum et est occaecati,1
5,repudiandae veniam quaerat sunt sed\nalias aut...,nesciunt quas odio,1


In [116]:
# I can access columns by passing the string label of column to df
df['title'][:5]

0    sunt aut facere repellat provident occaecati e...
1                                         qui est esse
2    ea molestias quasi exercitationem repellat qui...
3                                 eum et est occaecati
4                                   nesciunt quas odio
Name: title, dtype: object

I can also ask more interesting questions, for instance how may posts has each user made.  To do this I will group by user id, and then apply a function to the group.  This is akin to looking at the `userid` column and bucketing the rows, then applying a function to each bucket.  There are several built in functions, and I can write my own.

In [127]:
def group_size(group):
    return len(group)

user_counts = df.groupby('userId').apply(group_size)
# but count, sum, mean, etc. are common enough to have built in funtions.
# The bultin will count up every column for you (but as they are counts, they are all the same number)
user_counts2 = df.groupby('userId').count()

print user_counts.head()
print user_counts2.head()


userId
1         10
2         10
3         10
4         10
5         10
dtype: int64
        body  id  title  userId
userId                         
1         10  10     10      10
2         10  10     10      10
3         10  10     10      10
4         10  10     10      10
5         10  10     10      10

[5 rows x 4 columns]


That wasn't very intersting, but it's because we are using a toy web api for testing.

## Reading CSV files
You can also read local datasets to perform operations on.  CitiBike is a bike sharing program in the city, and they have made their ride information available.  Search for 'citibike api' or 'citibike system data'.  Get the data for December of last year (~12.1 MB) and unzip it.  When opening the file, you can specify a complete path.  If your ipython notebook and your file are in the same directory (folder), you can just specify the file name.  The file format is csv (comma separated values) and is a very simple file format.

Let's load the file, see how much data we have, and take a look at the first few rows.  This will load the csv into a pandas DataFrame:

In [142]:
df = pd.read_csv('201412-citibike-tripdata.csv')

In [143]:
print df.shape
df.head()

(399069, 15)


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1257,12/1/2014 00:00:28,12/1/2014 00:21:25,475,E 16 St & Irving Pl,40.735243,-73.987586,521,8 Ave & W 31 St,40.750450,-73.994811,16047,Customer,NaN,0
1,275,12/1/2014 00:00:43,12/1/2014 00:05:18,498,Broadway & W 32 St,40.748549,-73.988084,546,E 30 St & Park Ave S,40.744449,-73.983035,18472,Subscriber,1988,2
2,450,12/1/2014 00:01:22,12/1/2014 00:08:52,444,Broadway & W 24 St,40.742354,-73.989151,434,9 Ave & W 18 St,40.743174,-74.003664,19589,Subscriber,1983,1
3,1126,12/1/2014 00:02:17,12/1/2014 00:21:03,475,E 16 St & Irving Pl,40.735243,-73.987586,521,8 Ave & W 31 St,40.750450,-73.994811,21142,Customer,NaN,0
4,331,12/1/2014 00:02:21,12/1/2014 00:07:52,519,Pershing Square North,40.751873,-73.977706,527,E 33 St & 2 Ave,40.744023,-73.976056,18679,Subscriber,1986,2


That is an awful lot of rows.  In addition to the freedom you have with functions and transformations, another advantage to pandas is the ability to handle large data sets.  Trying to open this in your default spreadsheet program may crash the program.

All of this information is currently a basic data type.  Let's transform the startdate to datetimes, and then add them as a column.  This will make them easier to manipulate and group arbitrarily.

In [144]:
# import datetime
# def convert_to_date(x):
#     return datetime.datetime.strptime(x, "%m/%d/%Y %H:%M:%S")

# df['datetime'] = df['starttime'].apply(convert_to_date)
# df['date'] = df['datetime'].apply(lambda x: x.date)

df['date'] = df['starttime'].apply(lambda x: x.split()[0])
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date
0,1257,12/1/2014 00:00:28,12/1/2014 00:21:25,475,E 16 St & Irving Pl,40.735243,-73.987586,521,8 Ave & W 31 St,40.750450,-73.994811,16047,Customer,NaN,0,12/1/2014
1,275,12/1/2014 00:00:43,12/1/2014 00:05:18,498,Broadway & W 32 St,40.748549,-73.988084,546,E 30 St & Park Ave S,40.744449,-73.983035,18472,Subscriber,1988,2,12/1/2014
2,450,12/1/2014 00:01:22,12/1/2014 00:08:52,444,Broadway & W 24 St,40.742354,-73.989151,434,9 Ave & W 18 St,40.743174,-74.003664,19589,Subscriber,1983,1,12/1/2014
3,1126,12/1/2014 00:02:17,12/1/2014 00:21:03,475,E 16 St & Irving Pl,40.735243,-73.987586,521,8 Ave & W 31 St,40.750450,-73.994811,21142,Customer,NaN,0,12/1/2014
4,331,12/1/2014 00:02:21,12/1/2014 00:07:52,519,Pershing Square North,40.751873,-73.977706,527,E 33 St & 2 Ave,40.744023,-73.976056,18679,Subscriber,1986,2,12/1/2014


Now look at how many rides are taken each day.  We can do this by grouping by date, then taking the count.  Then we pick the appropriate column, in this case `tripduration`.  This will give us a Series (effectively a 1-d DataFrame)

In [149]:
s = df.groupby('date').count()['tripduration']
s.head()

date
12/1/2014     18519
12/10/2014    10399
12/11/2014    18141
12/12/2014    17892
12/13/2014    12549
Name: tripduration, dtype: int64

Now we have a list of values.  In fact, we can cast this as a list if we like:
```python
daily_trip_counts = list(s)
```
But this gets rid of a lot of the useful features that the Series structure has.  For instance the index that tells us on what date the count occured, or methods that let us find relavent summary statistics easily:

In [151]:
# Compute all the basic stats for a dataframe or series with the describe method
s.describe()

count       31.000000
mean     12873.193548
std       4854.495088
min       4047.000000
25%       9660.500000
50%      12549.000000
75%      17008.000000
max      21335.000000
Name: tripduration, dtype: float64

What about if we just want average trip length?  In this case there is no grouping or fancy transformation, we just want the mean of a column.  It is measured in seconds.

In [155]:
df['tripduration'].mean()

745.75784889329918